In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import os
import tqdm
import skimage.io
import glob
import datetime, os

from tqdm import tqdm

from skimage.io import imread, imshow
from skimage.transform import resize

from sklearn.utils import shuffle

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import InputLayer, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

%matplotlib inline

In [ ]:
%load_ext tensorboard

In [ ]:
import pickle
pickle_in = open('/content/drive/MyDrive/Dataset/X.pickle', "rb")
X = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('/content/drive/MyDrive/Dataset/y.pickle', "rb")
y = pickle.load(pickle_in)
pickle_in.close()

X.shape , y.shape

((3527, 128, 128, 3), (3527,))

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

rn50 = ResNet50(include_top=False, weights='imagenet', input_shape=(128,128,3))

In [ ]:
# Model Summary 

rn50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']      

In [ ]:
# We dont want to train all layers so, we do following step because we are using same weights given in VGG16

for layers in rn50.layers:
    layers.trainable = False

In [ ]:
# Introducing Flatten Layer

x = Flatten()(rn50.output)

In [ ]:
# Introducing FCC & Output Layer
fcc_layer_1 = Dense(units = 2048, activation = 'relu')(x)
dropout_1   = Dropout(0.3)(fcc_layer_1)

fcc_layer_2 = Dense(units = 1024, activation = 'relu')(dropout_1)
dropout_2   = Dropout(0.3)(fcc_layer_2)

final_layer = Dense(units = 1, activation = 'sigmoid')(dropout_2)

# Creating Final Model

model = Model(inputs = rn50.input, outputs = final_layer)

In [ ]:
# Model summary 

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']       

In [ ]:
# Model Compile 

model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Defining Callbacks

filepath = './content/drive/MyDrive/4th Year/Semester 1/CS464-2 (ML)/Project/Available Code/best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 50,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

learning_rate = ReduceLROnPlateau(monitor = 'val_accuracy',
                                  mode = 'max',
                                  patience = 10,
                                  factor = 0.3,
                                  min_delta = 0.00001)


callback_list = [earlystopping, checkpoint, learning_rate]

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# Fitting Model
X = preprocess_input(X)

model_history = model.fit(X,y,
                          batch_size = 128,
                          validation_split=0.2, 
                          epochs = 500,
                          verbose = 1,
                          callbacks = [tensorboard_callback])

Epoch 1/500
23/23 [==============================] - ETA: 0s - loss: 1.9062 - accuracy: 0.5750
Epoch 00001: val_accuracy improved from -inf to 0.65439, saving model to ./content/drive/MyDrive/4th Year/Semester 1/CS464-2 (ML)/Project/Available Code/best_weights.hdf5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


23/23 [==============================] - 17s 549ms/step - loss: 1.9062 - accuracy: 0.5750 - val_loss: 0.6462 - val_accuracy: 0.6544 - lr: 0.0010
Epoch 2/500
23/23 [==============================] - ETA: 0s - loss: 0.6398 - accuracy: 0.6852
Epoch 00002: val_accuracy did not improve from 0.65439
23/23 [==============================] - 8s 342ms/step - loss: 0.6398 - accuracy: 0.6852 - val_loss: 0.6448 - val_accuracy: 0.6544 - lr: 0.0010
Epoch 3/500
23/23 [==============================] - ETA: 0s - loss: 0.6287 - accuracy: 0.6856
Epoch 00003: val_accuracy did not improve from 0.65439
23/23 [==============================] - 8s 344ms/step - loss: 0.6287 - accuracy: 0.6856 - val_loss: 0.6487 - val_accuracy: 0.6544 - lr: 0.0010
Epoch 4/500
23/23 [==============================] - ETA: 0s - loss: 0.6342 - accuracy: 0.6852
Epoch 00004: val_accuracy did not improve from 0.65439
23/23 [==============================] - 8s 344ms/step - loss: 0.6342 - accuracy: 0.6852 - val_loss: 0.6556 - val_acc